In [58]:
"""
This notebook takes as input a csv file created from BigQuery (extract of an existing public dataset called 'ahr')
and transforms it to be used with other tables from 

Explanation of the values:   https://www.americashealthrankings.org/explore/measures/CVD
""";


In [12]:
# Imports

import pandas as pd

from eda_functions import data_exploration, draw_histogram, draw_countplot

pd.set_option('display.max_colwidth', None, 
              'display.max_columns', None,
              'display.max_rows', None)


In [3]:

data = pd.read_csv('bigquery_data.csv')

data_exploration(data)


,measure_name,state_name,subpopulation,value,source
0,Asthma,Arizona,NaN,10.0,"CDC, Behavioral Risk Factor Surveillance System"
1,Asthma,California,NaN,8.0,"CDC, Behavioral Risk Factor Surveillance System"
2,Asthma,Wyoming,NaN,9.0,"CDC, Behavioral Risk Factor Surveillance System"
3,Asthma,Colorado,NaN,9.0,"CDC, Behavioral Risk Factor Surveillance System"
4,Asthma,Georgia,NaN,8.0,"CDC, Behavioral Risk Factor Surveillance System"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18155 entries, 0 to 18154
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   measure_name   18155 non-null  object 
 1   state_name     18155 non-null  object 
 2   subpopulation  16647 non-null  object 
 3   value          16863 non-null  float64
 4   source         18147 non-null  object 
dtypes: float64(1), object(4)
memory usage: 709.3+ KB


Duplicates: 0 



,count,unique,top,freq,mean,std,min,25%,50%,75%,max
measure_name,18155,30,Poverty,832,NaN,NaN,NaN,NaN,NaN,NaN,NaN
state_name,18155,52,United States,356,NaN,NaN,NaN,NaN,NaN,NaN,NaN
subpopulation,16647,15,White,1353,NaN,NaN,NaN,NaN,NaN,NaN,NaN
value,16863.0,NaN,NaN,NaN,1738.28951,7505.76516,0.0,10.0,16.0,35.0,102684.0
source,18147,9,"CDC, Behavioral Risk Factor Surveillance System",10192,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
print(data['measure_name'].unique())
display(data.sort_values(by=['measure_name', 'state_name', 'subpopulation'])[:10])
# 'Cancer', 'Smoking', 'Diabetes', 'Depression', 'Per Capita Income', 'Excessive Drinking', 
# 'Physical Inactivity', 'Cardiovascular Diseases'

['Asthma' 'Cancer' 'Poverty' 'Smoking' 'Diabetes' 'Uninsured' 'Depression'
 'Able-Bodied' 'Unemployment' 'Child Poverty' 'Gender Pay Gap'
 'Flu Vaccination' 'Food Insecurity' 'Low Birthweight' 'Premature Death'
 'Infant Mortality' 'Income Inequality' 'Per Capita Income'
 'Excessive Drinking' 'High Health Status' 'Maternal Mortality'
 'Physical Inactivity' 'Cardiovascular Diseases' 'Residential Segregation'
 'Severe Housing Problems' 'Avoided Care Due to Cost'
 'Frequent Mental Distress' 'Multiple Chronic Conditions'
 'Dedicated Health Care Provider' 'Less Than High School Education']


,measure_name,state_name,subpopulation,value,source
17195,Able-Bodied,Alabama,American Indian/Alaska Native,72.0,"U.S. Census Bureau, American Community Survey PUMS"
14497,Able-Bodied,Alabama,Asian/Pacific Islander,92.0,"U.S. Census Bureau, American Community Survey PUMS"
15863,Able-Bodied,Alabama,Black/African American,79.0,"U.S. Census Bureau, American Community Survey PUMS"
9289,Able-Bodied,Alabama,College Grad,88.0,"U.S. Census Bureau, American Community Survey PUMS"
4469,Able-Bodied,Alabama,Female,80.0,"U.S. Census Bureau, American Community Survey PUMS"
11340,Able-Bodied,Alabama,High School Grad,77.0,"U.S. Census Bureau, American Community Survey PUMS"
5636,Able-Bodied,Alabama,Hispanic,90.0,"U.S. Census Bureau, American Community Survey PUMS"
12934,Able-Bodied,Alabama,Less Than High School,70.0,"U.S. Census Bureau, American Community Survey PUMS"
1896,Able-Bodied,Alabama,Male,80.0,"U.S. Census Bureau, American Community Survey PUMS"
12129,Able-Bodied,Alabama,Metropolitan Area,81.0,"U.S. Census Bureau, American Community Survey PUMS"


In [71]:
data_cardio = data[data['measure_name'] == 'Cardiovascular Diseases'].iloc[:, 1:4]
display(data_cardio[100:105])
data_cardio.dropna(inplace=True, subset=['subpopulation'])
data_cardio.reset_index(inplace=True, drop=True)
data_cardio = data_cardio.sort_values(by='value')

data_cardio = data_cardio.set_index(['state_name', 'subpopulation']).unstack(level=-1)
data_cardio.columns = data_cardio.columns.droplevel()
data_cardio = data_cardio[['Male', 'Female', 
                           'White', 'Black/African American', 'Hispanic', 
                           'American Indian/Alaska Native', 'Asian/Pacific Islander',
                               'Multiracial', 'Other Race', 
                           'College Grad', 'Some College', 'High School Grad',
                                'Less Than High School']].copy()
data_cardio.fillna(0, inplace=True)
# mask = data_cardio['subpopulation'] == 'Male'
# display(data_cardio[mask].tail(30))
data_exploration(data_cardio)

,state_name,subpopulation,value
2440,Arkansas,Male,14.0
2441,Kentucky,Male,14.0
2442,North Dakota,Male,9.0
2443,West Virginia,Male,16.0
3693,West Virginia,White,15.0


subpopulation,Male,Female,White,Black/African American,Hispanic,American Indian/Alaska Native,Asian/Pacific Islander,Multiracial,Other Race,College Grad,Some College,High School Grad,Less Than High School
state_name,,,,,,,,,,,,,
Alabama,14.0,11.0,14.0,10.0,0.0,19.0,0.0,14.0,15.0,8.0,11.0,13.0,21.0
Alaska,7.0,5.0,6.0,0.0,0.0,8.0,0.0,5.0,0.0,5.0,6.0,5.0,13.0
Arizona,10.0,7.0,10.0,7.0,5.0,7.0,0.0,7.0,12.0,7.0,9.0,8.0,10.0
Arkansas,14.0,11.0,13.0,10.0,7.0,20.0,0.0,16.0,26.0,7.0,12.0,12.0,21.0
California,8.0,5.0,8.0,7.0,5.0,10.0,5.0,9.0,9.0,5.0,8.0,6.0,8.0


<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, Alabama to Wyoming
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Male                           52 non-null     float64
 1   Female                         52 non-null     float64
 2   White                          52 non-null     float64
 3   Black/African American         52 non-null     float64
 4   Hispanic                       52 non-null     float64
 5   American Indian/Alaska Native  52 non-null     float64
 6   Asian/Pacific Islander         52 non-null     float64
 7   Multiracial                    52 non-null     float64
 8   Other Race                     52 non-null     float64
 9   College Grad                   52 non-null     float64
 10  Some College                   52 non-null     float64
 11  High School Grad               52 non-null     float64
 12  Less Than High School          52 non-null    

,count,mean,std,min,25%,50%,75%,max
subpopulation,,,,,,,,
Male,52.0,10.096154,2.012311,7.0,9.00,10.0,11.00,16.0
Female,52.0,7.519231,1.841679,5.0,6.00,7.0,8.00,14.0
White,52.0,9.346154,2.168261,3.0,8.00,9.0,10.25,15.0
Black/African American,52.0,6.884615,3.868697,0.0,6.00,8.0,10.00,12.0
Hispanic,52.0,4.653846,2.655993,0.0,4.00,5.0,6.00,9.0
American Indian/Alaska Native,52.0,12.942308,7.252892,0.0,9.75,14.0,18.00,26.0
Asian/Pacific Islander,52.0,1.134615,2.048709,0.0,0.00,0.0,2.00,7.0
Multiracial,52.0,9.211538,4.006546,0.0,7.00,10.0,11.25,17.0
Other Race,52.0,6.038462,6.862691,0.0,0.00,0.0,11.00,26.0


In [61]:
# data0 = data.pivot(index='state_name', columns=['measure_name', 'subpopulation'], values='value')
data0 = data.iloc[:, :-1].set_index(['measure_name', 'state_name', 'subpopulation']).unstack(level=-1)
data0.columns = data0.columns.droplevel()
display(data0.head())

# data_cardio = data_cardio.unstack()
# print(type(data_cardio))
# display(data_cardio)

subpopulation             NaN  American Indian/Alaska Native  \
measure_name state_name                                        
Able-Bodied  Alabama     80.0                           72.0   
             Alaska      85.0                           80.0   
             Arizona     84.0                           82.0   
             Arkansas    79.0                           75.0   
             California  87.0                           76.0   

subpopulation            Asian/Pacific Islander  Black/African American  \
measure_name state_name                                                   
Able-Bodied  Alabama                       92.0                    79.0   
             Alaska                        90.0                    85.0   
             Arizona                       92.0                    85.0   
             Arkansas                      92.0                    77.0   
             California                    91.0                    82.0   

subpopulation            College Grad  Female  High School Grad  Hispanic  \
measure_name state_name                                                     
Able-Bodied  Alabama             88.0    80.0              77.0      90.0   
             Alaska              90.0    86.0              83.0      89.0   
             Arizona             89.0    85.0              81.0      88.0   
             Arkansas            88.0    78.0              75.0      91.0   
             California          92.0    87.0              85.0      90.0   

subpopulation            Less Than High School  Male  Metropolitan Area  \
measure_name state_name                                                   
Able-Bodied  Alabama                      70.0  80.0               81.0   
             Alaska                       78.0  84.0               85.0   
             Arizona                      81.0  84.0               85.0   
             Arkansas                     70.0  79.0               81.0   
             California                   82.0  88.0               87.0   

subpopulation            Multiracial  Non-Metropolitan Area  Other Race  \
measure_name state_name                                                   
Able-Bodied  Alabama            76.0                   78.0        80.0   
             Alaska             84.0                   85.0         NaN   
             Arizona            84.0                   78.0        83.0   
             Arkansas           73.0                   75.0        87.0   
             California         86.0                   80.0        88.0   

subpopulation            Some College  White  
measure_name state_name                       
Able-Bodied  Alabama             83.0   80.0  
             Alaska              85.0   85.0  
             Arizona             84.0   82.0  
             Arkansas            81.0   78.0  
             California          87.0   84.0

In [ ]:

# data_cardio = data[data['measure_name'] == 'Cardiovascular Diseases'].iloc[:, 1:4]
# display(data_cardio[100:105])
# data_cardio.dropna(inplace=True, subset=['subpopulation'])
# data_cardio.reset_index(inplace=True, drop=True)
# data_cardio = data_cardio.sort_values(by='value')

# data_cardio = data_cardio.set_index(['state_name', 'subpopulation']).unstack(level=-1)
# data_cardio.columns = data_cardio.columns.droplevel()
# data_cardio = data_cardio[['Male', 'Female', 
#                            'White', 'Black/African American', 'Hispanic', 
#                            'American Indian/Alaska Native', 'Asian/Pacific Islander',
#                                'Multiracial', 'Other Race', 
#                            'College Grad', 'Some College', 'High School Grad',
#                                 'Less Than High School']].copy()
# data_cardio.fillna(0, inplace=True)
# # mask = data_cardio['subpopulation'] == 'Male'
# # display(data_cardio[mask].tail(30))
# data_exploration(data_cardio)
